# Countries Climate Change Segmentation

In [40]:
import pandas as pd

In [41]:
df_agriculture = pd.read_csv('data/02_df_agri_model.csv', index_col=0).rename({'Value': 'Agriculture_%'}, axis=1)
df_agriculture = df_agriculture.query('Indicators == "Agricultural land"')
df_agriculture['Agriculture_%'] = df_agriculture['Agriculture_%'].str.replace(' %', '').astype(float)
df_agriculture = df_agriculture.drop(columns='Indicators')
df = df_agriculture.copy()

In [42]:
df.query('Year == 2000')

,Year,Country,Agriculture_%
20,2000,Afghanistan,57.88
392,2000,Albania,41.75
...,...,...,...
74181,2000,Zambia,30.26
74536,2000,Zimbabwe,38.93


In [43]:
# df_fire = pd.read_csv('data/01_df_fires.csv', index_col=0).rename({'Value': 'Fire_Ha'}, axis=1)
# df_fire.Fire_Ha = df_fire.Fire_Ha.str.replace(' Ha', '').astype(float)
# df = df.merge(df_fire, how='left', on=['Year', 'Country'])

In [44]:
df_disasters = pd.read_csv('data/03_df_dis_model.csv', index_col=0).rename({'value': 'Disasters_N'}, axis=1)
df_disasters = df_disasters.drop(columns=['Indicator', 'Unit'])
df = df.merge(df_disasters, how='left')

In [45]:
df_emissions = pd.read_csv('data/04_df_emisiones_model.csv', index_col=0).rename({'value': 'Emissions'}, axis=1)
df_emissions = df_emissions.pivot_table(index=['Country', 'Year'], columns='Indicators', values='Emissions').reset_index()

In [46]:
df_emissions = df_emissions[['Country', 'Year', 'co2']].copy()
df = df.merge(df_emissions, how='left')

In [47]:
df_energy = pd.read_csv('data/05_df_ener_model.csv', index_col=0).rename({'value': 'Energy'}, axis=1)
df_energy = df_energy.query('Technology == "Fossil fuels" & Unit == "Megawatt (MW)"')
df_energy = df_energy.drop(columns=['Technology', 'Energy_Type', 'Unit'])
df = df.merge(df_energy, how='left')

In [48]:
df_health = pd.read_csv('data/06_df_health_indicators_model.csv', index_col=0).rename({
    'value': 'Health_Rate',
    'Country Name': 'Country'}, axis=1)
categories_health = ['Prevalence of overweight (% of adults)']
mask_health = df_health['Indicator Name'].isin(categories_health)
df_health = df_health[mask_health].copy()
df_health = df_health.pivot_table(index=['Country', 'Year'], columns='Indicator Name', values='Health_Rate').reset_index()
df = df.merge(df_health, how='left')

In [49]:
df_population = pd.read_csv('data/07_df_population_model.csv', index_col=0)
df_population = df_population.query('`Indicator Name` == "Urban population growth (annual %)"')
df_population = df_population.rename({
    'value': 'Urban Pop Anual Growth_%',
    'Country Name': 'Country'}, axis=1)
df_population = df_population.drop(columns=['Country Code', 'Indicator Name'])
df = df.merge(df_population, how='left')

In [50]:
df_temperature = pd.read_csv('data/08_df_temp_model.csv', index_col=0)
df = df.merge(df_temperature, how='left')

In [51]:
df_na = df.copy()

In [52]:
df_na.insert(0, 'NaNs', df_na.isna().sum(axis=1))

No tenemos datos de casi 40 paises en algunos años:

In [53]:
df_na_year = df_na.groupby('Year').NaNs.sum().reset_index()
df_na_year.assign(
    NaNs_rate = df_na_year.NaNs / 200
)

,Year,NaNs,NaNs_rate
0,1980,582,2.910
1,1981,585,2.925
...,...,...,...
39,2019,770,3.850
40,2020,762,3.810


No tenemos datos de casi 40 paises en algunos años:

In [54]:
df_na_country = df_na.groupby('Country').NaNs.sum().reset_index()
df_na_country.assign(
    NaNs_rate = df_na_country.NaNs / (2021 - 1980)
)

,Country,NaNs,NaNs_rate
0,Afghanistan,93,2.268293
1,Albania,60,1.463415
...,...,...,...
234,Zambia,52,1.268293
235,Zimbabwe,57,1.390244


In [55]:
df.shape

(8985, 9)

In [56]:
df.isna().sum()

Year                                         0
Country                                      0
Agriculture_%                                0
Disasters_N                               6361
co2                                       1877
Energy                                    5863
Prevalence of overweight (% of adults)    3311
Urban Pop Anual Growth_%                  1955
Average_Temperature                       3502
dtype: int64

In [57]:
df_group_years = df.groupby(['Year'])

In [58]:
df_year_2000 = df_group_years.get_group(2000)

In [59]:
df_year_2000.isna().sum()

Year                                        0
Country                                     0
Agriculture_%                               0
Disasters_N                               153
co2                                        44
Energy                                     80
Prevalence of overweight (% of adults)     65
Urban Pop Anual Growth_%                   48
Average_Temperature                        57
dtype: int64

In [60]:
df_na.pivot_table(index='Year', columns='Country', values=['NaNs'])

NaNs                                                \
Country Afghanistan Albania Algeria American Samoa Andorra Angola   
Year                                                                
1980            2.0     2.0     2.0            4.0     3.0    2.0   
1981            2.0     2.0     1.0            4.0     3.0    1.0   
...             ...     ...     ...            ...     ...    ...   
2019            4.0     3.0     2.0            4.0     4.0    2.0   
2020            4.0     3.0     3.0            4.0     4.0    2.0   

                                                                       \
Country Antigua and Barbuda Argentina Armenia Aruba Australia Austria   
Year                                                                    
1980                    3.0       1.0     NaN   3.0       1.0     1.0   
1981                    3.0       2.0     NaN   3.0       1.0     1.0   
...                     ...       ...     ...   ...       ...     ...   
2019                    3.0       2.0     4.0   4.0       2.0     2.0   
2020                    3.0       2.0     4.0   4.0       2.0     3.0   

                                                                        \
Country Azerbaijan Bahamas Bahrain Bangladesh Barbados Belarus Belgium   
Year                                                                     
1980           NaN     4.0     2.0        1.0      1.0     NaN     NaN   
1981           NaN     4.0     2.0        1.0      2.0     NaN     NaN   
...            ...     ...     ...        ...      ...     ...     ...   
2019           4.0     5.0     4.0        2.0      3.0     4.0     2.0   
2020           4.0     5.0     4.0        2.0      3.0     4.0     2.0   

                                                        \
Country Belgium-Luxembourg Belize Benin Bermuda Bhutan   
Year                                                     
1980                   6.0    2.0   1.0     4.0    2.0   
1981                   6.0    2.0   2.0     4.0    2.0   
...                    ...    ...   ...     ...    ...   
2019                   NaN    3.0   3.0     4.0    3.0   
2020                   NaN    2.0   2.0     4.0    3.0   

                                                                          \
Country Bolivia (Plurinational State of) Bosnia and Herzegovina Botswana   
Year                                                                       
1980                                 6.0                    NaN      2.0   
1981                                 6.0                    NaN      1.0   
...                                  ...                    ...      ...   
2019                                 6.0                    2.0      2.0   
2020                                 6.0                    2.0      3.0   

                                                                               \
Country Brazil British Virgin Islands Brunei Darussalam Bulgaria Burkina Faso   
Year                                                                            
1980       1.0                    3.0               4.0      2.0          1.0   
1981       1.0                    3.0               4.0      1.0          2.0   
...        ...                    ...               ...      ...          ...   
2019       2.0                    3.0               4.0      3.0          3.0   
2020       2.0                    3.0               4.0      3.0          2.0   

                                                                    \
Country Burundi Cabo Verde Cambodia Cameroon Canada Cayman Islands   
Year                                                                 
1980        2.0        3.0      2.0      2.0    1.0            4.0   
1981        2.0        4.0      2.0      2.0    1.0            4.0   
...         ...        ...      ...      ...    ...            ...   
2019        2.0        4.0      2.0      2.0    2.0            4.0   
2020        2.0        4.0      2.0      2.0    2.0            4.0   



In [61]:
df.to_csv('data/data_clean.csv', index=False)